In [61]:
import dash                     # pip install dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px     # pip install plotly==5.2.2

import numpy as np
import pandas as pd
import os
import glob
from datetime import datetime
# import timestamp
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce

from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures

import plotly.express as px

In [62]:

class Data():
    def __init__(self, path = None):
        
        self.directory = (glob.glob("/Data*merged.csv")) 
        # Add All the functionality to the class for organisation
        self.dailyNames = (glob.glob("daily*merged.csv")) 


        self.init()
        self.pdfs = []
        self.dailies = {}
    def init(self):
        for idx, dataName in enumerate(self.directory):
            self.pdfs.append(self.getDF(-1,dataName))

        for idx, data in enumerate(self.dailyNames):
            self.dailies[data]  =self.getDF(data)


    def getDF(self, i, byname =0):
        if(byname != 0):
            return(pd.read_csv(byname))
        return(pd.read_csv(self.directory[i]))





def containTimeInstant():
    timeList = ["time, "]

def read_file(path, expend_print = False, convertTimeDate = False):
    """ 
    read files
    """
    # formatting to pandas
    pd.Timestamp('2014-01-23 00:00:00', tz=None).to_pydatetime()

    print(path)
    
    # read csv
    df  = pd.read_csv(path)
    if(expend_print):
        print(path)
        print(f"Size: {df.shape}")
        print(df.head(10))
        print(df.columns)
        print((df.Id.unique()))
        print((list(df.columns)))
    df.dropna(subset=list(df.columns), how='any', axis=0).reset_index(inplace = True)

    if(convertTimeDate):
        pass

    ID = "Id"
    if ID in list(df.columns):
        # df['date_column'] = pd.to_datetime(df['datetime_column']).dt.date
        return df
    return None


def saved_file(df, name ):
    df.to_csv(f'{name}.csv')

def merge_data_to_one(files, safefile = False, saveFileName = "OutputMerged"):
    outputMerged = pd.DataFrame()
    
    for idx, fileName in enumerate(set(files)):
        data = read_file(fileName)
        if isinstance(data, pd.DataFrame):
            print(fileName)
            if(idx<1):
                outputMerged = data
            else:
                pd.merge(outputMerged, data, on = 'Id', how="left")
    
    if safefile: saved_file(outputMerged, saveFileName)


def compareSimilarity(A, B):
    A.sort()
    B.sort()

    max_val = max([len(A),len(B)])

    if max_val == len(A):arrMax, secondMax =A, B 
    else: arrMax, secondMax =A, B = A, B 
    
    sumCountSimilarities = 0

    sum_=0
    for i in list(arrMax):
        for j in list(secondMax):
            if i==j:
                if i<j : break
                sumCountSimilarities+=1
                break
             
    return (sumCountSimilarities, sumCountSimilarities/max_val)


def compareamountof(lists):
    l = len(lists)
    output = []

    for i in range(l):
        for j in range(i, l):
            A = read_file(lists[i])
            B = read_file(lists[j])


            similarities, ratiosim = compareSimilarity(A.Id.unique(), B.Id.unique())
            output.append([lists[i], lists[j],similarities, ratiosim])

    return output


def divideInsertTime(data, colName):
    # time_col_split = data.colName.str.split(" ")
    # split_amount = len(list(time_col_split[0]))

    date = pd.to_datetime(data[colName])
    try:
        Id_To_Idx = {id :num for num,id in enumerate(Heart_By_Idx.Id.unique())}
        Heart_By_Idx['PersonID'] = data.Id.map(Id_To_Idx)

        data["Hour"] = date.dt.hour
        data["Day"]  = date.dt.day 
        data["Minute"] = date.dt.minute
        data["DateOfWeek"] = date.dt.day_name()      
        print("Date Changed")
    except:
        e = sys.exc_info()
        print(f"Could Not Prase Data: Check Col Name; Erorr: {sys.exc_info()}")        

    return data




def pvalue_test(data, col_name, based_on_col_vals = None, tresh = 0.05):
    i = 0
    significant = []
    if based_on_col_vals ==None:
        print(data.columns)
        based_on_col_vals = data[col_name]
    print(f"Based on column: {col_name}")
    for (columnName, columnData) in data.iteritems():
        if i>1:
            stat, p = kendalltau(columnData, based_on_col_vals)
            print('Column Name : ', columnName)
            print(p)
            if(p<tresh):
                print(f"Significant - {p} - to {col_name} with {columnName}")
                significant.append((p,col_name, columnName))
        i +=1
    return significant


def sliceDataBasedonDescribeval(data, col,dateCol, tresh = 5):
    dis = data.describe()
    list_col = list((((dis.loc[col][dis.loc['std']>tresh]))).index)
    list_col.append(dateCol)

    return data[list_col]

def paired_tTest(data,col_1, check_col,population_mean,  tresh):
    
    # columns  
    cos = list(data.columns)

    
    val_mean =data[col_1].median()

    greater = totalMerge[totalMerge[col_1]>val_mean][check_col].values
    lower = totalMerge[totalMerge[col_1]<val_mean][check_col].values

    difference = greater-lower
    mean_diff = difference.mean()
    dif_std = difference.std()
    
    n = len(difference)

    t = (mean_diff - population_mean)/ (dif_std / np.sqrt(n))
    return(f"tValue: {t}, DF: {n-1}")
#TODO : Set Index: data.set_index("Id")

In [63]:
data_csvs = (glob.glob("../Data/*merged.csv")) 
dailyData = (glob.glob("../Data/daily*merged.csv"))  +  (glob.glob("../Data/heartrate*merged.csv")) 

data_csvs

['../Data\\dailyActivity_merged.csv',
 '../Data\\dailyCalories_merged.csv',
 '../Data\\dailyIntensities_merged.csv',
 '../Data\\dailySteps_merged.csv',
 '../Data\\heartrate_seconds_merged.csv',
 '../Data\\hourlyCalories_merged.csv',
 '../Data\\hourlyIntensities_merged.csv',
 '../Data\\hourlySteps_merged.csv',
 '../Data\\minuteCaloriesNarrow_merged.csv',
 '../Data\\minuteCaloriesWide_merged.csv',
 '../Data\\minuteIntensitiesNarrow_merged.csv',
 '../Data\\minuteIntensitiesWide_merged.csv',
 '../Data\\minuteMETsNarrow_merged.csv',
 '../Data\\minuteSleep_merged.csv',
 '../Data\\minuteStepsNarrow_merged.csv',
 '../Data\\minuteStepsWide_merged.csv',
 '../Data\\sleepDay_merged.csv',
 '../Data\\weightLogInfo_merged.csv']

In [171]:
# Data: https://www.dallasopendata.com/Services/Participants-Inventory/qgg6-h4bd

df = read_file(data_csvs[0])
df.ActivityDate = pd.to_datetime(df.ActivityDate)
df['Time_str'] = df.ActivityDate.astype('str')
# df["intake_time"] = df["intake_time"].dt.hour
# print(df.head())
df  =sliceDataBasedonDescribeval(df, 'std','ActivityDate')

df.ActivityDate.apply(lambda x: x.strftime('%Y%m%d'))
df['day_of_week'] = df.ActivityDate.apply(lambda x: x.strftime('%w'))
df.head()
df.columns

../Data\dailyActivity_merged.csv


Index(['Id', 'TotalSteps', 'VeryActiveMinutes', 'FairlyActiveMinutes',
       'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories', 'ActivityDate',
       'day_of_week'],
      dtype='object')

In [172]:
Id_To_Idx = {id :num for num,id in enumerate(df.Id.unique())}
Idx_to_Id = {num :id for num,id in enumerate(df.Id.unique())}
df['PersonID'] = df.Id.map(Id_To_Idx)


In [173]:
sleep = read_file(data_csvs[-2])
sleep.SleepDay = pd.to_datetime(sleep['SleepDay'])
sleep.rename(columns={"SleepDay": "Day"}, inplace=True)
sleep['DateStr'] = sleep.Day.astype('str')
df.head()


../Data\sleepDay_merged.csv


,Id,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,ActivityDate,day_of_week,PersonID
0,1503960366,13162,25,13,328,728,1985,2016-04-12,2,0
1,1503960366,10735,21,19,217,776,1797,2016-04-13,3,0
2,1503960366,10460,30,11,181,1218,1776,2016-04-14,4,0
3,1503960366,9762,29,34,209,726,1745,2016-04-15,5,0
4,1503960366,12669,36,10,221,773,1863,2016-04-16,6,0


In [67]:
steps = read_file(data_csvs[3])
steps.ActivityDay = pd.to_datetime(steps["ActivityDay"])
steps.rename(columns={"ActivityDay": "Day"}, inplace=True)
steps


../Data\dailySteps_merged.csv


,Id,Day,StepTotal
0,1503960366,2016-04-12,13162
1,1503960366,2016-04-13,10735
2,1503960366,2016-04-14,10460
3,1503960366,2016-04-15,9762
4,1503960366,2016-04-16,12669
...,...,...,...
935,8877689391,2016-05-08,10686
936,8877689391,2016-05-09,20226
937,8877689391,2016-05-10,10733
938,8877689391,2016-05-11,21420


In [68]:
calories = read_file(data_csvs[1])
calories.ActivityDay = pd.to_datetime(calories['ActivityDay'])
calories.rename(columns={"ActivityDay": "Day"}, inplace = True)
calories['DateStr'] = calories['Day'].astype('str')


meged_cl_sle =calories.merge(sleep, left_on=["Id","DateStr" ], right_on=["Id", "DateStr"]).drop(columns = ['Day_y'])
meged_cl_sle.rename(columns = {"Day_x": "Day"}, inplace=True)
# meged_cl_sle.drop(columns=["Id"], inplace= False).corr().round(2)
meged_cl_sle.head()



../Data\dailyCalories_merged.csv


,Id,Day,Calories,DateStr,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,2016-04-12,1985,2016-04-12,1,327,346
1,1503960366,2016-04-13,1797,2016-04-13,2,384,407
2,1503960366,2016-04-15,1745,2016-04-15,1,412,442
3,1503960366,2016-04-16,1863,2016-04-16,2,340,367
4,1503960366,2016-04-17,1728,2016-04-17,1,700,712


In [69]:
meged_cl_sle_steps =meged_cl_sle.merge(steps, left_on=["Id","Day" ], right_on=["Id", "Day"])

Id_To_Idx = {id :num for num,id in enumerate(meged_cl_sle_steps.Id.unique())}
Idx_to_Id = {num :id for num,id in enumerate(meged_cl_sle_steps.Id.unique())}
meged_cl_sle_steps['PersonID'] = meged_cl_sle_steps.Id.map(Id_To_Idx)
meged_cl_sle_steps.PersonID =  meged_cl_sle_steps.PersonID.astype('str')

In [70]:
active =read_file(data_csvs[0])
active.ActivityDate = pd.to_datetime(active.ActivityDate)
active.ActivityDate = active.ActivityDate.astype('str')


active_df  =sliceDataBasedonDescribeval(active, 'std','ActivityDate')

Id_To_Idx = {id :num for num,id in enumerate(active.Id.unique())}
Idx_to_Id = {num :id for num,id in enumerate(active.Id.unique())}
active_df['PersonID'] = active_df.Id.map(Id_To_Idx)


active_df.PersonID =  active_df.PersonID.astype('str')
active_df



meged_active_sleep  = active_df.merge(sleep, left_on=["Id","ActivityDate" ], right_on=["Id", "DateStr"]).drop(columns = ['ActivityDate'])
meged_active_sleep

../Data\dailyActivity_merged.csv


<ipython-input-70-67c0d163f48c>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

d:\Anaconda_2\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Id,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,PersonID,Day,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,DateStr
0,1503960366,13162,25,13,328,728,1985,0,2016-04-12,1,327,346,2016-04-12
1,1503960366,10735,21,19,217,776,1797,0,2016-04-13,2,384,407,2016-04-13
2,1503960366,9762,29,34,209,726,1745,0,2016-04-15,1,412,442,2016-04-15
3,1503960366,12669,36,10,221,773,1863,0,2016-04-16,2,340,367,2016-04-16
4,1503960366,9705,38,20,164,539,1728,0,2016-04-17,1,700,712,2016-04-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,8792009665,7174,10,20,301,749,2896,31,2016-04-30,1,343,360,2016-04-30
409,8792009665,1619,0,0,79,834,1962,31,2016-05-01,1,503,527,2016-05-01
410,8792009665,1831,0,0,101,916,2015,31,2016-05-02,1,415,423,2016-05-02
411,8792009665,2421,0,0,156,739,2297,31,2016-05-03,1,516,545,2016-05-03


In [71]:
meged_active_sleep

,Id,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,PersonID,Day,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,DateStr
0,1503960366,13162,25,13,328,728,1985,0,2016-04-12,1,327,346,2016-04-12
1,1503960366,10735,21,19,217,776,1797,0,2016-04-13,2,384,407,2016-04-13
2,1503960366,9762,29,34,209,726,1745,0,2016-04-15,1,412,442,2016-04-15
3,1503960366,12669,36,10,221,773,1863,0,2016-04-16,2,340,367,2016-04-16
4,1503960366,9705,38,20,164,539,1728,0,2016-04-17,1,700,712,2016-04-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,8792009665,7174,10,20,301,749,2896,31,2016-04-30,1,343,360,2016-04-30
409,8792009665,1619,0,0,79,834,1962,31,2016-05-01,1,503,527,2016-05-01
410,8792009665,1831,0,0,101,916,2015,31,2016-05-02,1,415,423,2016-05-02
411,8792009665,2421,0,0,156,739,2297,31,2016-05-03,1,516,545,2016-05-03


In [72]:
meged_active_sleep.head()

,Id,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,PersonID,Day,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,DateStr
0,1503960366,13162,25,13,328,728,1985,0,2016-04-12,1,327,346,2016-04-12
1,1503960366,10735,21,19,217,776,1797,0,2016-04-13,2,384,407,2016-04-13
2,1503960366,9762,29,34,209,726,1745,0,2016-04-15,1,412,442,2016-04-15
3,1503960366,12669,36,10,221,773,1863,0,2016-04-16,2,340,367,2016-04-16
4,1503960366,9705,38,20,164,539,1728,0,2016-04-17,1,700,712,2016-04-17


In [83]:
# df_ecdf = df[df["Calories"].isin(["DOG","CAT"])]
meged_active_sleep_gp = meged_active_sleep.groupby(['Id','Day'])[['TotalTimeInBed', 'Calories']].size()
fig_ecdf = px.line(meged_active_sleep, x="TotalTimeInBed", y="Calories", color="PersonID", markers=True)
fig_ecdf

## Heart Rate Model

In [74]:
doClean = False

heart_rate = read_file(dailyData[4])
heart_rate.Time = pd.to_datetime(heart_rate.Time)

if doClean:
    HeartRateFiltered = divideInsertTime(heart_rate, "Time")

heart_rate["Date"] = heart_rate.Time.dt.date
heart_rate["Time"]  = heart_rate["Time"].dt.time


heart_rate_std = heart_rate.groupby(["Date", "Id"]).std()
heart_rate_mean = heart_rate.groupby(["Date", "Id"]).mean()
heart_rate_median = heart_rate.groupby(["Date", "Id"]).median()


# compile the list of dataframes you want to merge
data_frames = [heart_rate_std, heart_rate_mean, heart_rate_median]
nan_value = 0

result_1 = pd.concat(data_frames, join='outer', axis=1).fillna(nan_value)

result_2 = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, 
                                              left_index=True, right_index=True, 
                                              how='outer'), 
                  data_frames).fillna(nan_value)


arr_idx = np.array([(list(values)[0],  list(values)[1]) for values in (result_2.index)])

result_2["Date"] = arr_idx[:,0]
result_2["Id"] = arr_idx[:,1]
result_2.rename(columns = {"Value_x": "Std","Value_y": "Mean", "Value": "Mediam"}, inplace = True)

Dropeed_Indx_HeartRate = result_2.reset_index(drop=True)



data_frames2 = [Dropeed_Indx_HeartRate, meged_cl_sle_steps]
result_3 = pd.concat(data_frames2, join='outer', axis=0).fillna(nan_value)

result_4 = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, 
                                              left_index=True, right_index=True, 
                                              how='outer'), 
                  data_frames).fillna(nan_value)

Dropeed_Indx_HeartRate.Date.astype('str')

meged_cl_sle_steps["DateSTR"]  =  meged_cl_sle_steps["Day"].astype('str')
Dropeed_Indx_HeartRate["TimeSTR"] = Dropeed_Indx_HeartRate.Date.astype('str')
totalMerge = meged_cl_sle_steps.merge(Dropeed_Indx_HeartRate, left_on=["Id","DateSTR" ], right_on=["Id", "TimeSTR"]).drop(columns=["Date", "TimeSTR", "DateSTR"])

../Data\heartrate_seconds_merged.csv


## Plot To Website

This is the very basic version. Scroll down to a more elaborated Website link

In [75]:
totalMerge.dtypes

Id                            object
Day                   datetime64[ns]
Calories                       int64
DateStr                       object
TotalSleepRecords              int64
TotalMinutesAsleep             int64
TotalTimeInBed                 int64
StepTotal                      int64
PersonID                      object
Std                          float64
Mean                         float64
Mediam                         int64
dtype: object

In [76]:
           # pip install pandas
initial = False
if initial:
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
    app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

    app.layout = html.Div([
        html.H1("What does the Body reveal", style={"textAlign":"center"}),
        html.Hr(),
        html.P("Choose Person of interest:"),
        html.Div(html.Div([
            dcc.Dropdown(id='Participant-type', clearable=False,
                        value="Std",
                        options=[{'label': x, 'value': x} for x in
                                df["Id"].unique()]),
        ],className="two columns"),className="row"),

        html.Div(id="output-div", children=[]),
    ])


    @app.callback(Output(component_id="output-div", component_property="children"),
                Input(component_id="Participant-type", component_property="value"),
    )
    def make_graphs(Participant_chosen):
        # HISTOGRAM

        df_hist = df[df["Id"]==Participant_chosen].copy()
        # df_hist_group_day = df_hist.groupby(['Day', 'Id']).mean()
        fig_hist = px.histogram(df_hist, x="TotalSteps", color="day_of_week")
        fig_hist.update_xaxes(categoryorder="total descending")


        # # Scatter CHART 
        sleepActive = meged_active_sleep.copy()
        
        sleepActive_par = sleepActive[sleepActive.Id ==Participant_chosen]
        sleepActive_par['Id'] = sleepActive_par['Id'].astype('str')
        fig_strip= px.scatter(sleepActive_par, x = 'TotalTimeInBed', y = 'SedentaryMinutes',symbol = 'Id',color = 'Id', marginal_x='histogram',marginal_y = 'box' ,trendline  ='ols', height  =600, title ='Sedentary Minutes vs Total time In Bed')

        # Line per participant with calories
        fig_ecdf = px.line(sleepActive_par, x="TotalTimeInBed", y="Calories", color="Id", markers=True)


        # Scatter General All Participants
        sleepActive_general = meged_active_sleep.copy()
        sleepActive_general['Id'] = sleepActive_general['Id'].astype('str')

        scatterGeneral= px.scatter(sleepActive_general, x = 'TotalTimeInBed', y = 'SedentaryMinutes',symbol = 'Id',color = 'Id', marginal_x='histogram',marginal_y = 'box' ,trendline  ='ols', height  =600, title ='Sedentary Minutes vs Total time In Bed')

        # # SUNBURST
        # df_sburst = df.dropna(subset=['chip_status'])
        # df_sburst = df_sburst[df_sburst["intake_type"].isin(["STRAY", "FOSTER", "OWNER SURRENDER"])]
        # fig_sunburst = px.sunburst(df_sburst, path=["Participant_type", "intake_type", "chip_status"])

        # # Empirical Cumulative Distribution
        # df_ecdf = df[df["Participant_type"].isin(["DOG","CAT"])]
        # fig_ecdf = px.ecdf(df_ecdf, x="TotalTimeInBed", color="Std")

        # # LINE CHART
        # df_line = df.sort_values(by=["intake_time"], ascending=True)
        # df_line = df_line.groupby(
        #     ["intake_time", "Participant_type"]).size().reset_index(name="count")
        # print(df_line.head())
        # fig_line = px.line(df_line, x="intake_time", y="count",
        #                    color="Participant_type", markers=True)

        return [

            html.H2("Selected Participants", style={"textAlign":"center"}),
            html.Hr(),
                html.Div([
                html.Div([dcc.Graph(figure=fig_hist)], className="four columns"),
                html.Div([dcc.Graph(figure=fig_strip)], className="four columns"),
                html.Div([dcc.Graph(figure=fig_ecdf)], className="four columns"),
            ], className="row"),

            html.H2("All Participants", style={"textAlign":"center"}),
                html.Div([
                html.Div([dcc.Graph(figure=scatterGeneral)], className="six columns"),
                # html.Div([dcc.Graph(figure=fig_ecdf)], className="six columns"),
            ], className="row"),

        ]


    if __name__ == '__main__':
        app.run_server(debug=False)


## Extended Dash Website 

Please run the following code to get the link

In [98]:
len(df.Id.unique())

33

In [105]:
Idx_to_Id

{0: 1503960366,
 1: 1624580081,
 2: 1644430081,
 3: 1844505072,
 4: 1927972279,
 5: 2022484408,
 6: 2026352035,
 7: 2320127002,
 8: 2347167796,
 9: 2873212765,
 10: 3372868164,
 11: 3977333714,
 12: 4020332650,
 13: 4057192912,
 14: 4319703577,
 15: 4388161847,
 16: 4445114986,
 17: 4558609924,
 18: 4702921684,
 19: 5553957443,
 20: 5577150313,
 21: 6117666160,
 22: 6290855005,
 23: 6775888955,
 24: 6962181067,
 25: 7007744171,
 26: 7086361926,
 27: 8053475328,
 28: 8253242879,
 29: 8378563200,
 30: 8583815059,
 31: 8792009665,
 32: 8877689391}

In [150]:
from jupyter_dash import JupyterDash     # pip install dash   
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input

from dash_extensions import Lottie       # pip install dash-extensions
import dash_bootstrap_components as dbc  # pip install dash-bootstrap-components
import plotly.express as px              # pip install plotly
import pandas as pd                      # pip install pandas
from datetime import date
import calendar
from wordcloud import WordCloud          # pip install wordcloud


# Lottie by Emil - https://github.com/thedirtyfew/dash-extensions
url_MeasureDays = "https://assets9.lottiefiles.com/private_files/lf30_5ttqPi.json"
url_caloriesBurned = "https://assets3.lottiefiles.com/packages/lf20_bdzc21c8.json"
url_sleep = "https://assets7.lottiefiles.com/private_files/lf30_em0dkba6.json"
url_PersonId = "https://assets9.lottiefiles.com/packages/lf20_dq0d7ah6.json"
url_Score = "https://assets4.lottiefiles.com/packages/lf20_rq3iw7zm.json"
options = dict(loop=True, autoplay=True, rendererSettings=dict(preserveAspectRatio='xMidYMid slice'))


# Bootstrap themes by Ann: https://hellodash.pythonanywhere.com/theme_explorer
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src='D:/github_/DataAnalysisCourse/Idan/PcturesWeb/Fbit.png', top=True) # 150px by 45px
            ],className='mb-2'),
            dbc.Card([
                dbc.CardBody([
                    dbc.CardLink("Data Set", target="_blank",
                                 href="https://github.com/idangrady/DataAnalysisCourse/tree/main/Data"
                    )
                ])
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.DatePickerSingle(
                        id='my-date-picker-start',
                        min_date_allowed= df.ActivityDate.min().date(),#date(2018, 1, 1),
                        max_date_allowed= df.ActivityDate.max().date(),
                        className='ml-5'
                    ),
                    dcc.DatePickerSingle(
                        id='my-date-picker-end',
                        min_date_allowed= df.ActivityDate.min().date(),#date(2018, 1, 1),
                        max_date_allowed= df.ActivityDate.max().date(),
                        className='ml-5'
                    ),

                 dcc.Dropdown(id='Participant-Id', 
                        clearable=True,
                        options=[{'label': x, 'value': x} for x in
                        df["PersonID"].unique()],
                        value="Id Participant",
                        className='mb-2 ml-2'
                    ),
                ])
            ], color="info"),
        ], width=8),
    ],className='mb-2 mt-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="60%", height="60%", url=url_MeasureDays)),
                dbc.CardBody([
                    html.H6('Measured Days'),
                    html.H2(id='Measured_Days', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="21%", height="21%", url=url_caloriesBurned)),
                dbc.CardBody([
                    html.H6('Average Calories Burned(daily)'),
                    html.H2(id='Calories_Burned', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="20%", height="20%", url=url_sleep)),
                dbc.CardBody([
                    html.H6('Average Sleep duration (min)'),
                    html.H2(id='Ave_SleepDuration', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="35%", height="35%", url=url_PersonId)),
                dbc.CardBody([
                    html.H6('Perosn Id'),
                    html.H2(id='Perosn_Id', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="32%", height="32%", url=url_Score)),
                dbc.CardBody([
                    html.H6('Score'),
                    html.H2(id='Score_Prediction', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
    ],className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='line-chart', figure={}),
                ])
            ]),
        ], width=6),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='Scatter_graph', figure={}),
                ])
            ]),
        ], width=4),
    ],className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='STDGraph-scatter', figure={}),
                ])
            ]),
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='corr_Graph', figure={}),
                ])
            ]),
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='wordcloud', figure={}),
                ])
            ]),
        ], width=4),
    ],className='mb-2'),
], fluid=True)





# Updating the 5 number cards
@app.callback(
    Output('Measured_Days','children'),
    Output('Calories_Burned','children'),
    Output('Ave_SleepDuration','children'),
    Output('Perosn_Id','children'),
    Output('Score_Prediction','children'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('Participant-Id','value'),
)
def update_small_cards(start_date, end_date, Participant_chosen): # add person Id
    # Connections
    # Par_Id_str = str(Participant_chosen)
    print(id)
    print(type(id))
    given = False
    if id is not None:
        given = True
    
    Participant_chosen = int(Idx_to_Id(Participant_chosen))

    dff_c = sleep.copy()

    dff_c = dff_c[(dff_c['Day']>=start_date) & (dff_c['Day']<=end_date)]  #&(sleep['Id']==parId)
    if given ==True:
        dff_c = dff_c[(dff_c['Day']>=start_date) & (dff_c['Day']<=end_date)&(sleep['Id']==Participant_chosen)]  #

    # dff_c.groupby(['Id']).aggregate(['mean', 'count','std'])['TotalTimeInBed']
    mean_perDay = round(dff_c.TotalTimeInBed.mean(), 4)
    # num_recorded_days = dff_c.count()[0]



    df_calories = calories.copy()
    df_calories_date = df_calories[(df_calories['Day']>=start_date) & (df_calories['Day']<=end_date)] 

    if given ==True:
        df_calories_date = df_calories[(df_calories['Day']>=start_date) & (df_calories['Day']<=end_date)&(sleep['Id']==Participant_chosen)] 


    # sum_calories = df_calories_date.Calories.sum()
    sum_calories = round(df_calories_date.groupby('Day')['Calories'].mean().mean(), 4) 
    
    person_id = None if not given else Id_To_Idx[Participant_chosen]


    # get the number of days from a different more 
    dff_c = df[(df['ActivityDate']>=start_date) & (df['ActivityDate']<=end_date)].copy()  #&(sleep['Id']==parId)
    if givenPar ==True:
        dff_c = dff_c[(dff_c['ActivityDate']>=start_date) & (dff_c['ActivityDate']<=end_date)&(dff_c['Id']==Participant_chosen)]  #

    conctns_num = len(dff_c.groupby('ActivityDate'))

    return conctns_num,  sum_calories, mean_perDay, person_id, "--"


@app.callback(
    Output('line-chart','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('Participant-Id','value'),
)
def update_personlisedWeeklyStepsGraph(start_date, end_date, Participant_chosen):
        # check if participant is given
        givenPar = None if not Participant_chosen else True
        Participant_chosen =  int(Idx_to_Id(Participant_chosen))

        print(givenPar)
        print(type(Participant_chosen))
        
        dff_c = df[(df['ActivityDate']>=start_date) & (df['ActivityDate']<=end_date)].copy()  #&(sleep['Id']==parId)
        if givenPar ==True:
            dff_c = dff_c[(dff_c['ActivityDate']>=start_date) & (dff_c['ActivityDate']<=end_date)&(dff_c['Id']==Participant_chosen)]  #

        # Create Graph
        fig_hist = px.histogram(dff_c, x="TotalSteps", color="day_of_week")
        fig_hist.update_xaxes(categoryorder="total descending")

        return fig_hist


@app.callback(
    Output('Scatter_graph','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('Participant-Id','value'),
)
def update_personlisedScatterGraph(start_date, end_date, Participant_chosen):
        # check if participant is given
        givenPar = None if not Participant_chosen else True
        Participant_chosen =  int(Idx_to_Id(Participant_chosen))

        sleepActive_par = meged_active_sleep[(meged_active_sleep['Day']>=start_date) & (meged_active_sleep['Day']<=end_date)].copy()
        if givenPar ==True:
            sleepActive_par = meged_active_sleep[(meged_active_sleep['Day']>=start_date) & (meged_active_sleep['Day']<=end_date)&(meged_active_sleep['Id']==Participant_chosen)].copy()

        # Create Graph
        sleepActive_par['Id'] = sleepActive_par['Id'].astype('str')
        fig_strip= px.scatter(sleepActive_par, x = 'TotalTimeInBed', y = 'SedentaryMinutes',symbol = 'Id',color = 'Id', marginal_x='histogram',marginal_y = 'box' ,trendline  ='ols', height  =600, title ='Sedentary Minutes vs Total time In Bed')
        # fig_strip
        return fig_strip

@app.callback(
    Output('STDGraph-scatter','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('Participant-Id','value'),
)

def plotSTD_perParticipants(start_date, end_date, Participant_chosen):

    # Check wether Pariticapnt is given
    givenPar = None if not Participant_chosen else True

    Participant_chosen =  int(Idx_to_Id(Participant_chosen))
    mean_asleep = totalMerge.TotalTimeInBed.median()
    aboveMedian =totalMerge[(totalMerge['Day']>=start_date) & (totalMerge['Day']<=end_date) & (totalMerge['TotalTimeInBed']>mean_asleep)]['Std'].values
    belowMedian =totalMerge[(totalMerge['Day']>=start_date) & (totalMerge['Day']<=end_date) & (totalMerge['TotalTimeInBed']<mean_asleep)]['Std'].values

    if givenPar:
        aboveMedian =totalMerge[(totalMerge['Day']>=start_date) & (totalMerge['Day']<=end_date) & (totalMerge['TotalTimeInBed']>mean_asleep) & (totalMerge['Id']==Participant_chosen)]['Std'].values
        belowMedian =totalMerge[(totalMerge['Day']>=start_date) & (totalMerge['Day']<=end_date) & (totalMerge['TotalTimeInBed']<mean_asleep)& (totalMerge['Id']==Participant_chosen)]['Std'].values


    min_length = min(len(aboveMedian), len(belowMedian))
    belowMedian = belowMedian[:min_length]
    aboveMedian = aboveMedian[:min_length]

    new_date  =aboveMedian-belowMedian
    # Lagends
    x = ['Above Median', 'Below Median']
    import plotly.graph_objects as px2
    df = px.data.tips()

    # Make the plot
    fig = px2.Figure()
    fig.add_trace(px2.Box(y=aboveMedian, quartilemethod="linear", name="STD Above Median", boxmean='sd'))
    fig.add_trace(px2.Box(y=belowMedian, quartilemethod="inclusive", name="STD Below Median", boxmean='sd'))
    # fig.add_trace(go.Box(y=data, quartilemethod="exclusive", name="Exclusive Quartile Mode"))
    fig.update_traces(boxpoints='all', jitter=0)
    fig.update_layout(
        yaxis_title='STD Heart Rate (Daily)',
        title = 'Distribution Of Daily STD Heart Rate Based On Sleeping Length (2 Groups)',
        # boxmode='group' # group together boxes of the different traces for each value of x
    )

    # Return value of Fig
    return fig



# plot total corralation prt time and participant


@app.callback(
    Output('corr_Graph','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('Participant-Id','value'),
)

def plotCorrGraph(start_date, end_date, Participant_chosen):
    
    print(type(Participant_chosen))
    # Check wether Pariticapnt is given
    givenPar = None if not Participant_chosen else True
    Participant_chosen = int(Idx_to_Id(int(Participant_chosen)))
    print(Participant_chosen)
    print(type(Participant_chosen))
    # print(total_merge.Id))
    total_merge =total_merge.copy()
    df =total_merge[(total_merge['Day']>=start_date) & (total_merge['Day']<=end_date)]

    if givenPar:
        df =totalMerge[(totalMerge['Day']>=start_date) & (totalMerge['Day']<=end_date) & (totalMerge['Id']==Participant_chosen)]


    fig = px.scatter_matrix(df, dimensions=["Calories", "StepTotal", "Std", "Mean","Mediam"], color="PersonID")
    return fig


if __name__=='__main__':
    app.run_server(debug=False, port=8000)

 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Jun/2022 23:28:01] "GET /_alive_808c0f3b-6993-456b-afc8-d23142ac405b HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8000/


127.0.0.1 - - [01/Jun/2022 23:28:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 23:28:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 23:28:04] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 23:28:04] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jun/2022 23:28:04] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jun/2022 23:28:04] "GET /_dash-component-suites/dash_extensions/async-lottie.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jun/2022 23:28:04] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jun/2022 23:28:04] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jun/2022 23:28:04] "GET /_favicon.ico?v=2.4.1 HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2022 23:28:04] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Jun/2022 23:28:04] "POST /_dash-update-component HTTP/1.1" 5

<built-in function id>
<class 'builtin_function_or_method'>
<class 'str'>


127.0.0.1 - - [01/Jun/2022 23:28:05] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Jun/2022 23:28:05] "POST /_dash-update-component HTTP/1.1" 500 -


<built-in function id>
<class 'builtin_function_or_method'>
<class 'NoneType'>


127.0.0.1 - - [01/Jun/2022 23:28:05] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Jun/2022 23:28:05] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Jun/2022 23:28:05] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Jun/2022 23:28:06] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Jun/2022 23:28:06] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Jun/2022 23:28:06] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Jun/2022 23:28:06] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Jun/2022 23:28:07] "POST /_dash-update-component HTTP/1.1" 500 -


<built-in function id>
<class 'builtin_function_or_method'>
<class 'NoneType'>


127.0.0.1 - - [01/Jun/2022 23:28:08] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Jun/2022 23:28:08] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Jun/2022 23:28:08] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Jun/2022 23:28:08] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Jun/2022 23:28:08] "POST /_dash-update-component HTTP/1.1" 500 -


<built-in function id>
<class 'builtin_function_or_method'>
<class 'NoneType'>


In [199]:
meged_active_sleep.head()

NameError: name 'sleepActive' is not defined

In [282]:
from jupyter_dash import JupyterDash     # pip install dash   
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input

from dash_extensions import Lottie       # pip install dash-extensions
import dash_bootstrap_components as dbc  # pip install dash-bootstrap-components
import plotly.express as px              # pip install plotly
import pandas as pd                      # pip install pandas
from datetime import date
import calendar
from wordcloud import WordCloud          # pip install wordcloud


# Lottie by Emil - https://github.com/thedirtyfew/dash-extensions
url_MeasureDays = "https://assets9.lottiefiles.com/private_files/lf30_5ttqPi.json"
url_caloriesBurned = "https://assets3.lottiefiles.com/packages/lf20_bdzc21c8.json"
url_sleep = "https://assets7.lottiefiles.com/private_files/lf30_em0dkba6.json"
url_PersonId = "https://assets9.lottiefiles.com/packages/lf20_dq0d7ah6.json"
url_Score = "https://assets4.lottiefiles.com/packages/lf20_rq3iw7zm.json"
options = dict(loop=True, autoplay=True, rendererSettings=dict(preserveAspectRatio='xMidYMid slice'))


# Bootstrap themes by Ann: https://hellodash.pythonanywhere.com/theme_explorer
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src='D:/github_/DataAnalysisCourse/Idan/PcturesWeb/Fbit.png', top=True) # 150px by 45px
            ],className='mb-2'),
            dbc.Card([
                dbc.CardBody([
                    dbc.CardLink("Data Set", target="_blank",
                                 href="https://github.com/idangrady/DataAnalysisCourse/tree/main/Data"
                    )
                ])
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.DatePickerSingle(
                        id='my-date-picker-start',
                        min_date_allowed= df.ActivityDate.min().date(),#date(2018, 1, 1),
                        max_date_allowed= df.ActivityDate.max().date(),
                        className='ml-5'
                    ),
                    dcc.DatePickerSingle(
                        id='my-date-picker-end',
                        min_date_allowed= df.ActivityDate.min().date(),#date(2018, 1, 1),
                        max_date_allowed= df.ActivityDate.max().date(),
                        className='ml-5'
                    ),

                 dcc.Dropdown(id='Participant-Id', 
                        clearable=True,
                        options=[{'label': x, 'value': x} for x in
                        df["Id"].unique()],
                        value="Id Participant",
                        className='mb-2 ml-2'
                    ),
                ])
            ], color="info"),
        ], width=8),
    ],className='mb-2 mt-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="60%", height="60%", url=url_MeasureDays)),
                dbc.CardBody([
                    html.H6('Measured Days'),
                    html.H2(id='Measured_Days', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="21%", height="21%", url=url_caloriesBurned)),
                dbc.CardBody([
                    html.H6('Total Calories Burned'),
                    html.H2(id='Calories_Burned', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="20%", height="20%", url=url_sleep)),
                dbc.CardBody([
                    html.H6('Average Sleep duration'),
                    html.H2(id='Ave_SleepDuration', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="35%", height="35%", url=url_PersonId)),
                dbc.CardBody([
                    html.H6('Perosn Id'),
                    html.H2(id='Perosn_Id', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="32%", height="32%", url=url_Score)),
                dbc.CardBody([
                    html.H6('Score'),
                    html.H2(id='Score_Prediction', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
    ],className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='line-chart', figure={}),
                ])
            ]),
        ], width=6),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='Scatter_graph', figure={}),
                ])
            ]),
        ], width=4),
    ],className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='STDGraph-scatter', figure={}),
                ])
            ]),
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='corr_Graph', figure={}),
                ])
            ]),
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='continoustime', figure={}),
                ])
            ]),
        ], width=4),
    ],className='mb-2'),
], fluid=True)





# Updating the 5 number cards
@app.callback(
    Output('Measured_Days','children'),
    Output('Calories_Burned','children'),
    Output('Ave_SleepDuration','children'),
    Output('Perosn_Id','children'),
    Output('Score_Prediction','children'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('Participant-Id','value'),
)
def update_small_cards(start_date, end_date, id): # add person Id
    # Connections
    Par_Id_str = str(id)
    print(id)
    print(type(id))
    given = False
    if id is not None:
        given = True
    dff_c = sleep.copy()
    dff_c.Id = dff_c.Id.astype('str')

    dff_c = dff_c[(dff_c['Day']>=start_date) & (dff_c['Day']<=end_date)]  #&(sleep['Id']==parId)
    if given ==True:
        dff_c = dff_c[(dff_c['Day']>=start_date) & (dff_c['Day']<=end_date)&(sleep['Id']==Par_Id_str)]  #

    # dff_c.groupby(['Id']).aggregate(['mean', 'count','std'])['TotalTimeInBed']
    mean_perDay = round(dff_c.TotalTimeInBed.mean(), 4)
    # num_recorded_days = dff_c.count()[0]



    df_calories = calories.copy()
    df_calories_date = df_calories[(df_calories['Day']>=start_date) & (df_calories['Day']<=end_date)] 

    if given ==True:
        df_calories_date = df_calories[(df_calories['Day']>=start_date) & (df_calories['Day']<=end_date)&(sleep['Id']==Par_Id_str)] 


    # sum_calories = df_calories_date.Calories.sum()
    sum_calories = round(df_calories_date.groupby('Day')['Calories'].mean().mean(), 4) 
    
    person_id = None if not given else Id_To_Idx[Par_Id_str]


    # get the number of days from a different more 
    dff_c = df[(df['ActivityDate']>=start_date) & (df['ActivityDate']<=end_date)].copy()  #&(sleep['Id']==parId)
    if givenPar ==True:
        dff_c = dff_c[(dff_c['ActivityDate']>=start_date) & (dff_c['ActivityDate']<=end_date)&(dff_c['Id']==Participant_chosen)]  #

    conctns_num = len(dff_c.groupby('ActivityDate'))

    return conctns_num,  sum_calories, mean_perDay, person_id, "--"


@app.callback(
    Output('line-chart','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('Participant-Id','value'),
)
def update_personlisedWeeklyStepsGraph(start_date, end_date, Participant_chosen):
        # check if participant is given
        givenPar = None if not Participant_chosen else True
        print(givenPar)
        print(type(Participant_chosen))
        
        dff_c = df[(df['ActivityDate']>=start_date) & (df['ActivityDate']<=end_date)].copy()  #&(sleep['Id']==parId)
        if givenPar ==True:
            dff_c = dff_c[(dff_c['ActivityDate']>=start_date) & (dff_c['ActivityDate']<=end_date)&(dff_c['Id']==Participant_chosen)]  #

        # Create Graph
        fig_hist = px.histogram(dff_c, x="TotalSteps", color="day_of_week")
        fig_hist.update_xaxes(categoryorder="total descending")

        return fig_hist


@app.callback(
    Output('Scatter_graph','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('Participant-Id','value'),
)
def update_personlisedScatterGraph(start_date, end_date, Participant_chosen):
        # check if participant is given
        givenPar = None if not Participant_chosen else True

        sleepActive_par = meged_active_sleep[(meged_active_sleep['Day']>=start_date) & (meged_active_sleep['Day']<=end_date)].copy()
        if givenPar ==True:
            sleepActive_par = meged_active_sleep[(meged_active_sleep['Day']>=start_date) & (meged_active_sleep['Day']<=end_date)&(meged_active_sleep['Id']==Participant_chosen)].copy()

        # Create Graph
        sleepActive_par['Id'] = sleepActive_par['Id'].astype('str')
        fig_strip= px.scatter(sleepActive_par, x = 'TotalTimeInBed', y = 'SedentaryMinutes',symbol = 'Id',color = 'Id', marginal_x='histogram',marginal_y = 'box' ,trendline  ='ols', height  =600, title ='Sedentary Minutes vs Total time In Bed')
        # fig_strip
        return fig_strip

@app.callback(
    Output('STDGraph-scatter','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('Participant-Id','value'),
)

def plotSTD_perParticipants(start_date, end_date, Participant_chosen):

    # Check wether Pariticapnt is given
    givenPar = None if not Participant_chosen else True

    
    mean_asleep = totalMerge.TotalTimeInBed.median()
    aboveMedian =totalMerge[(totalMerge['Day']>=start_date) & (totalMerge['Day']<=end_date) & (totalMerge['TotalTimeInBed']>mean_asleep)]['Std'].values
    belowMedian =totalMerge[(totalMerge['Day']>=start_date) & (totalMerge['Day']<=end_date) & (totalMerge['TotalTimeInBed']<mean_asleep)]['Std'].values

    if givenPar:
        aboveMedian =totalMerge[(totalMerge['Day']>=start_date) & (totalMerge['Day']<=end_date) & (totalMerge['TotalTimeInBed']>mean_asleep) & (totalMerge['Id']==Participant_chosen)]['Std'].values
        belowMedian =totalMerge[(totalMerge['Day']>=start_date) & (totalMerge['Day']<=end_date) & (totalMerge['TotalTimeInBed']<mean_asleep)& (totalMerge['Id']==Participant_chosen)]['Std'].values


    min_length = min(len(aboveMedian), len(belowMedian))
    belowMedian = belowMedian[:min_length]
    aboveMedian = aboveMedian[:min_length]

    new_date  =aboveMedian-belowMedian
    # Lagends
    x = ['Above Median', 'Below Median']
    import plotly.graph_objects as px2
    df = px.data.tips()



    # Make the plot
    fig = px2.Figure()
    fig.add_trace(px2.Box(y=aboveMedian, quartilemethod="linear", name="STD Above Median", boxmean='sd'))
    fig.add_trace(px2.Box(y=belowMedian, quartilemethod="inclusive", name="STD Below Median", boxmean='sd'))
    # fig.add_trace(go.Box(y=data, quartilemethod="exclusive", name="Exclusive Quartile Mode"))
    fig.update_traces(boxpoints='all', jitter=0)
    fig.update_layout(
        yaxis_title='STD Heart Rate (Daily)',
        title = 'Distribution Of Daily STD Heart Rate Based On Sleeping Length (2 Groups)',
        # boxmode='group' # group together boxes of the different traces for each value of x
    )


    # Return value of Fig
    return fig




@app.callback(
    Output('corr_Graph','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('Participant-Id','value'),
)

def plotCorrGraph(start_date, end_date, Participant_chosen):
    
    # Check wether Pariticapnt is given
    givenPar = None if not Participant_chosen else True

    # print(total_merge.Id))
    total_merge =totalMerge.copy()
    df =total_merge[(total_merge['Day']>=start_date) & (total_merge['Day']<=end_date)]

    if givenPar:
        df =totalMerge[(totalMerge['Day']>=start_date) & (totalMerge['Day']<=end_date) & (totalMerge['Id']==Participant_chosen)]

    fig = px.scatter_matrix(df, dimensions=["Calories", "StepTotal", "Std", "Mean","Mediam"], color="PersonID")
    return fig


@app.callback(
    Output('continoustime','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('Participant-Id','value'),
)

def plotCorrGraph(start_date, end_date, Participant_chosen):
    
    # Check wether Pariticapnt is given
    givenPar = None if not Participant_chosen else True

    # print(total_merge.Id))
    total_merge =meged_active_sleep.copy()

    meged_cat =total_merge[(total_merge['Day']>=start_date) & (total_merge['Day']<=end_date)]

    if givenPar:
        meged_cat =total_merge[(total_merge['Day']>=start_date) & (total_merge['Day']<=end_date) & (total_merge['Id']==Participant_chosen)]

    # df =df.groupby('Id').mean()
    # fig = px.parallel_categories(df, color="TotalTimeInBed", color_continuous_scale=px.colors.sequential.Inferno)




    labels_ = ["low", "weak meduim","medium", "Weak high" ,"high"]
    meged_cat["totalStepCat"]= pd.cut(np.array(meged_cat.TotalSteps.values),len(labels_),labels=labels_)
    meged_cat["VeryActiveMinutesCat"]= pd.cut(np.array(meged_cat.VeryActiveMinutes.values),len(labels_),labels=labels_)
    meged_cat["SedentaryMinutesCat"]= pd.cut(np.array(meged_cat.SedentaryMinutes.values),len(labels_),labels=labels_)
    meged_cat["TotalMinutesAsleepCat"]= pd.cut(np.array(meged_cat.TotalMinutesAsleep.values),len(labels_),labels=labels_)

    meged_cat= meged_cat[['Id','totalStepCat', 'VeryActiveMinutesCat', 'SedentaryMinutesCat','TotalMinutesAsleepCat']]


    # meged_cat.Id = meged_cat.Id.astype('str')

    # pd.cut(np.array([1, 7, 5, 4, 6, 3]), 3, labels=["low", "medium", "high"])
    fig = px.parallel_categories(meged_cat, color_continuous_scale=px.colors.sequential.Inferno)

    return fig


# d = totalMerge.groupby('Id').mean()
# d
# fig = 



if __name__=='__main__':
    app.run_server(debug=False, port=8000)

 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Jun/2022 01:17:54] "GET /_alive_808c0f3b-6993-456b-afc8-d23142ac405b HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8000/


127.0.0.1 - - [02/Jun/2022 01:18:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2022 01:18:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2022 01:18:06] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2022 01:18:07] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Jun/2022 01:18:07] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Jun/2022 01:18:07] "GET /_dash-component-suites/dash_extensions/async-lottie.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Jun/2022 01:18:07] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Jun/2022 01:18:07] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Jun/2022 01:18:07] "GET /_favicon.ico?v=2.4.1 HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2022 01:18:07] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [02/Jun/2022 01:18:07] "POST /_dash-update-component HTTP/1.1" 5

Id ParticipantTrue
<class 'str'>

<class 'str'>


127.0.0.1 - - [02/Jun/2022 01:18:07] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [02/Jun/2022 01:18:07] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [02/Jun/2022 01:18:07] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [02/Jun/2022 01:18:07] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [02/Jun/2022 01:18:07] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [02/Jun/2022 01:18:07] "POST /_dash-update-component HTTP/1.1" 500 -


...

127.0.0.1 - - [02/Jun/2022 01:18:11] "POST /_dash-update-component HTTP/1.1" 200 -
<ipython-input-282-a840ca88d054>:361: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-282-a840ca88d054>:362: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-282-a840ca88d054>:363: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

1503960366
<class 'int'>
True
<class 'int'>


<ipython-input-282-a840ca88d054>:184: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

127.0.0.1 - - [02/Jun/2022 01:18:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2022 01:18:25] "POST /_dash-update-component HTTP/1.1" 200 -
<ipython-input-282-a840ca88d054>:361: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-282-a840ca88d054>:362: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-282-a840ca88d054>:363: SettingWithCopyWarning:


1503960366True
<class 'int'>

<class 'int'>


<ipython-input-282-a840ca88d054>:184: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-282-a840ca88d054>:361: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

127.0.0.1 - - [02/Jun/2022 01:18:51] "POST /_dash-update-component HTTP/1.1" 200 -
<ipython-input-282-a840ca88d054>:362: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-282-a840ca88d054>:363: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [170]:
df

,sepal_length,sepal_width,petal_length,petal_width,species,species_id
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,3
146,6.3,2.5,5.0,1.9,virginica,3
147,6.5,3.0,5.2,2.0,virginica,3
148,6.2,3.4,5.4,2.3,virginica,3


In [54]:
mean_asleep = totalMerge.TotalTimeInBed.median()
aboveMedian =totalMerge[(totalMerge['Day']>=start_date) & (totalMerge['Day']<=end_date) & (totalMerge['TotalTimeInBed']>mean_asleep)]['Std'].values
belowMedian =totalMerge[(totalMerge['Day']>=start_date) & (totalMerge['Day']<=end_date) & (totalMerge['TotalTimeInBed']<mean_asleep)]['Std'].values
len(belowMedian)

min_length = min(len(aboveMedian), len(belowMedian))
min_length

11

In [89]:
meged_active_sleep

,Id,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,PersonID,Day,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,DateStr
0,1503960366,13162,25,13,328,728,1985,0,2016-04-12,1,327,346,2016-04-12
1,1503960366,10735,21,19,217,776,1797,0,2016-04-13,2,384,407,2016-04-13
2,1503960366,9762,29,34,209,726,1745,0,2016-04-15,1,412,442,2016-04-15
3,1503960366,12669,36,10,221,773,1863,0,2016-04-16,2,340,367,2016-04-16
4,1503960366,9705,38,20,164,539,1728,0,2016-04-17,1,700,712,2016-04-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,8792009665,7174,10,20,301,749,2896,31,2016-04-30,1,343,360,2016-04-30
409,8792009665,1619,0,0,79,834,1962,31,2016-05-01,1,503,527,2016-05-01
410,8792009665,1831,0,0,101,916,2015,31,2016-05-02,1,415,423,2016-05-02
411,8792009665,2421,0,0,156,739,2297,31,2016-05-03,1,516,545,2016-05-03


In [129]:
Id_To_Idx[2026352035]
print(Idx_to_Id[Id_To_Idx[2026352035]])

2026352035


In [139]:
totalMerge

,Id,Day,Calories,DateStr,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,StepTotal,PersonID,Std,Mean,Mediam
0,2026352035,2016-04-17,1214,2016-04-17,1,437,498,838,4,3.486183,68.656250,68
1,2026352035,2016-04-25,1576,2016-04-25,1,506,531,6017,4,7.662667,99.505814,98
2,2026352035,2016-05-02,1690,2016-05-02,1,511,543,7018,4,9.091549,84.134571,81
3,2026352035,2016-05-09,1869,2016-05-09,1,531,556,10685,4,9.533803,98.233901,98
4,2347167796,2016-04-13,2038,2016-04-13,1,467,531,10352,6,13.014521,73.812905,71
...,...,...,...,...,...,...,...,...,...,...,...,...
177,8792009665,2016-04-30,2896,2016-04-30,1,343,360,7174,23,14.072832,81.464642,81
178,8792009665,2016-05-01,1962,2016-05-01,1,503,527,1619,23,9.892940,67.810171,66
179,8792009665,2016-05-02,2015,2016-05-02,1,415,423,1831,23,10.397178,67.403919,65
180,8792009665,2016-05-03,2297,2016-05-03,1,516,545,2421,23,12.616910,74.371940,74


In [144]:
totalMerge[(totalMerge['Id']==(2026352035))]


,Id,Day,Calories,DateStr,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,StepTotal,PersonID,Std,Mean,Mediam
0,2026352035,2016-04-17,1214,2016-04-17,1,437,498,838,4,3.486183,68.656250,68
1,2026352035,2016-04-25,1576,2016-04-25,1,506,531,6017,4,7.662667,99.505814,98
2,2026352035,2016-05-02,1690,2016-05-02,1,511,543,7018,4,9.091549,84.134571,81
3,2026352035,2016-05-09,1869,2016-05-09,1,531,556,10685,4,9.533803,98.233901,98


In [154]:
start_date = '2016-04-12'
end_date = '2016-05-15'
Participant_chosen= "1503960366"
given = False
givenPar = False
i = 2
print(str(Idx_to_Id[6]))




2026352035


NameError: name 'total_merge' is not defined

,Calories,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,StepTotal,Std,Mean,Mediam
Id,,,,,,,,
2026352035,1587.250000,1.000000,496.250000,532.000000,6139.500000,7.443551,87.632634,86.250000
2347167796,1971.333333,1.000000,446.800000,491.333333,8533.200000,12.223268,74.230514,71.666667
4020332650,3195.000000,1.000000,349.375000,379.750000,6596.750000,15.071863,82.201731,82.625000
4388161847,3131.625000,1.291667,403.125000,426.208333,10974.708333,14.375829,65.733764,62.000000
4558609924,2197.000000,1.000000,127.600000,140.000000,8139.000000,16.076566,86.653252,84.800000
5553957443,1875.677419,1.225806,463.483871,505.870968,8612.580645,11.954509,67.774545,63.967742
5577150313,3539.192308,1.038462,432.000000,460.615385,9260.076923,19.910421,68.937479,62.192308
6117666160,2460.833333,1.222222,478.777778,510.166667,8823.833333,13.198422,82.057550,82.000000
6775888955,2344.666667,1.000000,349.666667,369.000000,3499.000000,14.393304,80.860774,76.333333


In [168]:

# fig = px.parallel_coordinates(totalMerge, color="TotalTimeInBed", labels={"Mediam": "Mediam",
#                   "Calories": "Sepal Width", "StepTotal": "StepTotal",
#                    },
#                     color_continuous_scale=px.colors.diverging.Tealrose, color_continuous_midpoint=2)


fig.show()

In [263]:
meged_active_sleep.head()

,Id,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,PersonID,Day,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,DateStr
0,1503960366,13162,25,13,328,728,1985,0,2016-04-12,1,327,346,2016-04-12
1,1503960366,10735,21,19,217,776,1797,0,2016-04-13,2,384,407,2016-04-13
2,1503960366,9762,29,34,209,726,1745,0,2016-04-15,1,412,442,2016-04-15
3,1503960366,12669,36,10,221,773,1863,0,2016-04-16,2,340,367,2016-04-16
4,1503960366,9705,38,20,164,539,1728,0,2016-04-17,1,700,712,2016-04-17


In [281]:
meged_cat = meged_active_sleep.copy()
labels_ = ["low", "weak meduim","medium", "Weak high" ,"high"]
meged_cat["totalStepCat"]= pd.cut(np.array(meged_cat.TotalSteps.values),len(labels_),labels=labels_)
meged_cat["VeryActiveMinutesCat"]= pd.cut(np.array(meged_cat.VeryActiveMinutes.values),len(labels_),labels=labels_)
meged_cat["SedentaryMinutesCat"]= pd.cut(np.array(meged_cat.SedentaryMinutes.values),len(labels_),labels=labels_)
meged_cat["TotalMinutesAsleepCat"]= pd.cut(np.array(meged_cat.TotalMinutesAsleep.values),len(labels_),labels=labels_)

meged_cat= meged_cat[['Id','totalStepCat', 'VeryActiveMinutesCat', 'SedentaryMinutesCat','TotalMinutesAsleepCat']]


# meged_cat.Id = meged_cat.Id.astype('str')

# pd.cut(np.array([1, 7, 5, 4, 6, 3]), 3, labels=["low", "medium", "high"])
fig = px.parallel_categories(meged_cat, color_continuous_scale=px.colors.sequential.Inferno)
fig

,totalStepCat,VeryActiveMinutesCat,SedentaryMinutesCat,TotalMinutesAsleepCat
0,medium,low,medium,medium
1,medium,low,medium,medium
2,medium,low,medium,medium
3,medium,low,medium,medium
4,medium,low,medium,high
...,...,...,...,...
408,low,low,medium,medium
409,low,low,medium,medium
410,low,low,high,medium
411,low,low,medium,medium


In [230]:
class_dim = go.parcats.Dimension(
    values=meged_active_sleep.Id,
    categoryorder='Id', label="Class"
)

colorscale = [[0, 'lightsteelblue'], [1, 'mediumseagreen']];

color = meged_active_sleep.Id;

fig = go.Figure(data = [go.Parcats(dimensions=[class_dim],
        line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        # labelfont={'size': 18, 'family': 'Times'},
        # tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

ValueError: 
    Invalid value of type 'builtins.str' received for the 'categoryorder' property of parcats.dimension
        Received value: 'Id'

    The 'categoryorder' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['trace', 'category ascending', 'category descending',
            'array']

In [231]:
titanic_df.Pclass

0      3
1      1
2      3
3      1
4      3
      ..
151    1
152    3
153    3
154    3
155    1
Name: Pclass, Length: 156, dtype: int64

In [228]:
import plotly.graph_objects as go


titanic_df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/titanic.csv")

# Create dimensions
class_dim = go.parcats.Dimension(
    values=titanic_df.Pclass,
    categoryorder='category ascending', label="Class"
)

gender_dim = go.parcats.Dimension(values=titanic_df.Sex, label="Gender")

survival_dim = go.parcats.Dimension(
    values=titanic_df.Survived, label="Outcome", categoryarray=[0, 1],
    ticktext=['perished', 'survived']
)

# Create parcats trace
color = titanic_df.Survived;
colorscale = [[0, 'lightsteelblue'], [1, 'mediumseagreen']];

fig = go.Figure(data = [go.Parcats(dimensions=[class_dim, gender_dim, survival_dim],
        line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        # labelfont={'size': 18, 'family': 'Times'},
        # tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()